# RAW Image Enhancement - Quick Start Tutorial

This notebook demonstrates how to use the RAW image enhancement system.

**Topics Covered:**
1. Loading trained models
2. Processing RAW images
3. Running inference
4. Visualizing results
5. Performance benchmarking

## 1. Setup and Imports

In [ ]:
import sys
sys.path.append('..')

import torch
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Import project modules
from models.raw_diffusion_unet import RAWVAE
from models.consistency_distillation import ConsistencyModel
from models.optical_flow import RAWOpticalFlow, AlignmentModule
from inference.realtime_pipeline import RealTimePipeline
from data.raw_loader import load_dng_file

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

## 2. Load Trained Models

In [ ]:
# Configuration
device = 'cuda' if torch.cuda.is_available() else 'cpu'
checkpoint_dir = Path('../outputs')

print("Loading models...")

# Load VAE
vae = RAWVAE(in_channels=4, latent_channels=16, channels=64, num_res_blocks=2)
vae_checkpoint = torch.load(checkpoint_dir / 'vae_final.pt', map_location=device)
vae.load_state_dict(vae_checkpoint['model_state_dict'])
vae.to(device).eval()

print("✓ VAE loaded")

# Load Optical Flow
flow_net = RAWOpticalFlow(in_channels=4, feature_dim=128, num_levels=4)
flow_checkpoint = torch.load(checkpoint_dir / 'flow_final.pt', map_location=device)
flow_net.load_state_dict(flow_checkpoint['model_state_dict'])
alignment = AlignmentModule(flow_net)
alignment.to(device).eval()

print("✓ Optical Flow loaded")

# Load Consistency Model
consistency = ConsistencyModel(in_channels=16, model_channels=128, num_res_blocks=2)
consistency_checkpoint = torch.load(checkpoint_dir / 'consistency_final.pt', map_location=device)
consistency.load_state_dict(consistency_checkpoint['model_state_dict'])
consistency.to(device).eval()

print("✓ Consistency Model loaded")

# Create inference pipeline
pipeline = RealTimePipeline(
    vae=vae,
    consistency_model=consistency,
    alignment=alignment,
    device=device,
    num_inference_steps=2,
    use_adaptive_steps=True
)

print("\n✓ All models loaded successfully!")
print(f"  Device: {device}")
print(f"  Inference steps: 2")
print(f"  Adaptive steps: True")

## 3. Load Sample RAW Burst

In [ ]:
# Load burst of DNG files
data_dir = Path('../data/test')
burst_files = sorted(data_dir.glob('burst_001/*.dng'))[:8]

if len(burst_files) == 0:
    print("⚠️  No DNG files found. Creating synthetic test data...")
    
    # Create synthetic burst for demo
    burst = torch.randn(1, 8, 4, 512, 512, device=device)
    print(f"Created synthetic burst: {burst.shape}")
else:
    print(f"Loading {len(burst_files)} frames...")
    
    burst_images = []
    for file in burst_files:
        img = load_dng_file(str(file))
        burst_images.append(img)
    
    # Stack into burst tensor
    burst = torch.stack(burst_images, dim=0).unsqueeze(0).to(device)
    print(f"Loaded burst: {burst.shape}")

# Visualize first frame
first_frame = burst[0, 0].cpu().numpy()

plt.figure(figsize=(12, 4))
for i in range(4):
    plt.subplot(1, 4, i+1)
    plt.imshow(first_frame[i], cmap='gray')
    plt.title(f'Bayer Channel {i}')
    plt.axis('off')
plt.tight_layout()
plt.show()

## 4. Run Inference

In [ ]:
print("Running inference...")

import time

# Warmup
with torch.no_grad():
    _ = pipeline.forward(burst[:, :4], return_intermediate=False)

# Actual inference
torch.cuda.synchronize() if torch.cuda.is_available() else None
start_time = time.time()

with torch.no_grad():
    results = pipeline.forward(burst, return_intermediate=True)

torch.cuda.synchronize() if torch.cuda.is_available() else None
latency = (time.time() - start_time) * 1000  # ms

print(f"\n✓ Inference complete!")
print(f"  Latency: {latency:.2f} ms")
print(f"  Steps used: {results['num_steps']}")
print(f"  Output shape: {results['enhanced'].shape}")

# Extract results
enhanced = results['enhanced']
if 'aligned' in results:
    aligned = results['aligned']
if 'latent' in results:
    latent = results['latent']

## 5. Visualize Results

In [ ]:
# Compare input vs output
input_frame = burst[0, 0].cpu().numpy()
output_frame = enhanced[0].cpu().numpy()

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

# Input channels
for i in range(4):
    axes[0, i].imshow(input_frame[i], cmap='gray')
    axes[0, i].set_title(f'Input Ch{i}')
    axes[0, i].axis('off')

# Output channels
for i in range(4):
    axes[1, i].imshow(output_frame[i], cmap='gray')
    axes[1, i].set_title(f'Enhanced Ch{i}')
    axes[1, i].axis('off')

plt.suptitle('Input vs Enhanced Comparison', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Convert to RGB for better visualization
def bayer_to_rgb(bayer_tensor):
    """Simple Bayer to RGB conversion."""
    r = bayer_tensor[0]
    g = (bayer_tensor[1] + bayer_tensor[2]) / 2
    b = bayer_tensor[3]
    return np.stack([r, g, b], axis=-1)

input_rgb = bayer_to_rgb(input_frame)
output_rgb = bayer_to_rgb(output_frame)

plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.imshow(np.clip(input_rgb, 0, 1))
plt.title('Input (First Frame)', fontsize=14, fontweight='bold')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(np.clip(output_rgb, 0, 1))
plt.title('Enhanced (Multi-frame Fusion)', fontsize=14, fontweight='bold')
plt.axis('off')

plt.tight_layout()
plt.show()

print("Notice the noise reduction and detail enhancement in the output!")

## 6. Performance Benchmarking

In [ ]:
# Benchmark different configurations
print("Running benchmark...")

results = pipeline.benchmark(
    burst_sizes=[(512, 512)],
    burst_lengths=[8],
    num_iterations=10,
    warmup_iterations=2
)

print("\nBenchmark Results:")
print("="*50)
for key, value in results.items():
    if isinstance(value, dict):
        print(f"\n{key}:")
        for k, v in value.items():
            if isinstance(v, (int, float)):
                print(f"  {k}: {v:.2f}")
            else:
                print(f"  {k}: {v}")
    else:
        print(f"{key}: {value}")

## 7. Model Information

In [ ]:
# Get model sizes
model_sizes = pipeline.get_model_size()

print("Model Information:")
print("="*50)
for name, size_mb in model_sizes.items():
    print(f"{name}: {size_mb:.2f} MB")

total_size = sum(model_sizes.values())
print(f"\nTotal: {total_size:.2f} MB")

# Memory usage
if torch.cuda.is_available():
    memory_allocated = torch.cuda.memory_allocated() / (1024 ** 2)
    memory_reserved = torch.cuda.memory_reserved() / (1024 ** 2)
    
    print(f"\nGPU Memory:")
    print(f"  Allocated: {memory_allocated:.2f} MB")
    print(f"  Reserved: {memory_reserved:.2f} MB")

## 8. Export Results

In [ ]:
# Save enhanced image
output_dir = Path('../notebook_outputs')
output_dir.mkdir(exist_ok=True)

# Save as numpy array
np.save(output_dir / 'enhanced.npy', output_frame)

# Save RGB visualization
from PIL import Image
rgb_uint8 = (np.clip(output_rgb, 0, 1) * 255).astype(np.uint8)
Image.fromarray(rgb_uint8).save(output_dir / 'enhanced_rgb.png')

print(f"✓ Results saved to {output_dir}")
print(f"  - enhanced.npy (RAW data)")
print(f"  - enhanced_rgb.png (visualization)")

## Summary

In this notebook, we:
1. ✅ Loaded pre-trained models (VAE, Optical Flow, Consistency)
2. ✅ Processed RAW burst sequences
3. ✅ Ran real-time inference (<30ms)
4. ✅ Visualized input vs enhanced output
5. ✅ Benchmarked performance
6. ✅ Exported results

### Next Steps:
- Try with your own RAW burst sequences
- Experiment with different inference step counts
- Fine-tune models on your specific dataset
- Deploy to production using the API

### Resources:
- Full Documentation: `../README.md`
- Training Guide: `../TRAINING_DEPLOYMENT.md`
- API Documentation: `../api/serve.py`